## Ejemplo de uso

- conda environment python = 3.11
- install chromadb
- install crewai-tools

In [1]:
from crewai_tools import CSVSearchTool

In [2]:
import os
import yaml
from crewai import Agent, Task, Crew, Process, LLM

In [ ]:
# Cargar la API Key de OpenAI
os.environ["OPENAI_API_KEY"] = "XXX"
os.environ['OPENAI_MODEL_NAME'] = 'gpt-4.1-mini'

Ejemplo general de prueba

In [ ]:
# Paso 1: Crear los agentes

# Agente 1: Investigador
investigador = Agent(
    role="Investigador de temas",
    goal="Buscar información precisa y actualizada sobre el tema solicitado",
    backstory="Es un experto en búsqueda en internet y recolección de datos fiables.",
    verbose=False
)

# Agente 2: Redactor
redactor = Agent(
    role="Redactor técnico",
    goal="Escribir un resumen claro y profesional con base en la información recopilada",
    backstory="Especialista en redacción de contenido técnico y divulgación.",
    verbose=False
)

In [ ]:
# Paso 2: Definir las tareas

# Tarea 1: Búsqueda de información
tarea_busqueda = Task(
    description="Investiga sobre los últimos avances en energía solar. Incluye descubrimientos recientes, tecnologías emergentes y tendencias del mercado.",
    expected_output="Un documento con los datos clave encontrados",
    agent=investigador
)

# Tarea 2: Redacción del resumen
tarea_resumen = Task(
    description="Usa la información del investigador para crear un resumen ejecutivo de máximo 300 palabras sobre los avances en energía solar.",
    expected_output="Un resumen claro, profesional y de fácil comprensión",
    agent=redactor
)

In [ ]:
# Crew (equipo)
equipo = Crew(
    agents=[investigador, redactor],
    tasks=[tarea_busqueda, tarea_resumen],
    verbose=False
)

In [ ]:
# Ejecutar la colaboración entre agentes
resultado = equipo.kickoff()


In [ ]:
print("\n📝 Resultado final:\n")
print(resultado)

## Caso de estudio puntual

Ejemplo definiendo un agente por cada tipo de ticket

In [ ]:
import os
from crewai import Agent, Task, Crew

In [ ]:
# Sample ticket
ticket = {
    "ticket_id": "1001",
    "customer_message": "My laptop won't connect to WiFi and I need help ASAP."
}

In [ ]:
# Step 1: Define Agents
supervisor = Agent(
    role="Ticket Classifier",
    goal="Classify customer support tickets into IT, Project, or Marketing topics.",
    backstory="A senior supervisor who accurately identifies the topic of customer issues.",
    verbose=False
)

it_agent = Agent(
    role="IT Support Agent",
    goal="Resolve IT issues reported by customers.",
    backstory="An IT technician who provides fast and effective support for technical problems.",
    verbose=False
)

project_agent = Agent(
    role="Project Agent",
    goal="Handle project-related inquiries from customers.",
    backstory="An experienced project manager who supports clients on timelines and tasks.",
    verbose=False
)

marketing_agent = Agent(
    role="Marketing Agent",
    goal="Respond to marketing-related concerns from customers.",
    backstory="A skilled marketing professional who communicates campaign details clearly.",
    verbose=False
)


In [ ]:
# Step 2: Define Tasks

# Task 1: Classification by supervisor
classification_task = Task(
    description=(
        f"You are the supervisor. Classify the following ticket into one of these categories: "
        f"'IT', 'Project', or 'Marketing'.\n\n"
        f"Customer message: {ticket['customer_message']}"
    ),
    expected_output="Only respond with one word: IT, Project, or Marketing.",
    agent=supervisor
)

In [ ]:
# We will run this first to get the classification
crew1 = Crew(
    agents=[supervisor],
    tasks=[classification_task],
    verbose=False
)

classification_result = str(crew1.kickoff()).strip().lower()


In [ ]:

# Step 3: Dynamically choose specialized task based on classification

if "it" in classification_result:
    specialized_task = Task(
        description=f"Respond to this IT-related customer issue:\n\n{ticket['customer_message']}",
        expected_output="A detailed and helpful response to resolve the customer's IT issue.",
        agent=it_agent
    )
elif "project" in classification_result:
    specialized_task = Task(
        description=f"Respond to this project-related customer concern:\n\n{ticket['customer_message']}",
        expected_output="A professional answer addressing the customer's project concern.",
        agent=project_agent
    )
elif "marketing" in classification_result:
    specialized_task = Task(
        description=f"Respond to this marketing-related customer message:\n\n{ticket['customer_message']}",
        expected_output="A clear and informative marketing response.",
        agent=marketing_agent
    )
else:
    print("❌ Could not classify ticket. No specialized task executed.")
    specialized_task = None

# Step 4: Execute specialized task
if specialized_task:
    crew2 = Crew(
        agents=[specialized_task.agent],
        tasks=[specialized_task],
        verbose=False
    )
    final_response = crew2.kickoff()

    print(f"\n🧾 Ticket ID: {ticket['ticket_id']}")
    print(f"📂 Classification: {classification_result.capitalize()}")
    print(f"💬 Agent Response:\n{final_response}")



🧾 Ticket ID: 1001
📂 Classification: It
💬 Agent Response:
I'm sorry to hear that your laptop isn't connecting to WiFi. Let's go through some detailed troubleshooting steps to get your connection working again as quickly as possible:

1. **Check WiFi is turned on:**
   - On your laptop, ensure the WiFi is enabled. You can usually do this by clicking the WiFi icon in the system tray (usually bottom-right corner on Windows, top-right on macOS).
   - Also, some laptops have a physical WiFi switch or a function key (like Fn + F2) to toggle WiFi – make sure it is on.

2. **Restart your laptop and router/modem:**
   - Sometimes simply rebooting your devices resolves connectivity issues.
   - Turn off your laptop, unplug your router and modem from power for about 30 seconds, then plug them back in.
   - After the router/modem fully powers back on, turn your laptop back on and try connecting again.

3. **Forget and reconnect to the WiFi network:**
   - On your laptop, go to the list of WiFi net

Ejemplo definiendo una arquitectura secuencial, un solo agente que analice cualquier ticket y realice varias actividades con ese ticket

In [ ]:
# Simula lectura de CSV con una lista de tickets (id y descripción)
tickets = [
    {"ticket_id": "1001", "description": "My laptop won't connect to WiFi and I need urgent help."},
    {"ticket_id": "1002", "description": "I want to know the status of project X and the delivery date."},
    {"ticket_id": "1003", "description": "I did not receive the last invoice and I need it to be sent to me."},
]

In [ ]:
# Agents

reader_agent = Agent(
    name="ReaderAgent",
    role="Read tickets from CSV and send description to classifier agent",
    goal="Read and submit tickets one by one for classification",
    backstory="Extract tickets from a database or CSV for processing in the support chain.",
    verbose=False
)

classifier_agent = Agent(
    name="ClassifierAgent",
    role="Classify tickets into categories",
    goal="Assign to each ticket a category: Technical Support, Billing, Project, Marketing, Other.",
    backstory="Specialist in classifying tickets to direct them to the right agent.",
    verbose=False
)

specialist_agent = Agent(
    name="SpecialistAgent",
    role="Analyze ticket and generate response",
    goal=(
        "Determine priority, extract keywords, analyze problem and write response "
        "professional and empathetic to the client."
    ),
    backstory="Agent specialized in deep ticket analysis and response generation.",
    verbose=False
)

In [ ]:
# Función para procesar un ticket completo
def procesar_ticket(ticket):

    # 1. Clasificación
    task_clasify = Task(
        description=(
            f"Classify the following ticket into a category (Technical Support, Billing, Project, Marketing, Other):\n\n"
            f"Ticket: \"{ticket['description']}\""
        ),
        expected_output="A single suggested category.",
        agent=classifier_agent
    )
    crew_classify = Crew(agents=[classifier_agent], tasks=[task_clasify], verbose=False)
    categoria = str(crew_classify.kickoff()).strip()

    # 2. Análisis + respuesta con agente especialista
    task_specialist = Task(
        description=(
            f"For the next ticket with category '{categoria}':\n\n"
            f"\"{ticket['description']}\"\n\n"
            "Performs the following:\n"
            "1. Determines the priority level: Critical, Medium or Low.\n"
            "2. Extract the most important keywords.\n"
            "3. Analyze the main problem.\n"
            "4. Write a professional, clear and empathetic response to the client.\n\n"
            "Returns the information in this exact format:\n"
            "Priority: <level>\n"
            "Keywords: <list>\n"
            "Analysis: <short analysis>\n"
            "Answer: <text in the answer>\n"
        ),
        expected_output="Text with priority, keywords, analysis and response.",
        agent=specialist_agent
    )
    crew_specialist = Crew(agents=[specialist_agent], tasks=[task_specialist], verbose=False)
    analisis_y_respuesta = str(crew_specialist.kickoff()).strip()

In [ ]:
# Imprimir resultado completo
print(f"\n🧾 Ticket ID: {ticket['ticket_id']}")
print(f"📂 Category of the ticket: {categoria}")
print(analisis_y_respuesta)
print("---------------------------------------------------")

# Simular que el reader_agent pasa cada ticket
for ticket in tickets:
    procesar_ticket(ticket)


🧾 Ticket ID: 1001
📂 Category of the ticket: Technical Support
Priority: Critical
Keywords: laptop, won't connect, WiFi, urgent help
Analysis: The client is experiencing a complete inability to connect their laptop to WiFi, which is likely preventing them from working or accessing important services. Since they emphasize urgency, this issue is impacting their productivity or critical tasks and needs prompt resolution.
Answer: Dear Customer,

I’m sorry to hear that your laptop is unable to connect to WiFi. I understand how important it is to have a reliable internet connection, especially when you need urgent access. Let’s work together to get this resolved quickly.

Please try the following steps:
1. Restart your laptop and your WiFi router/modem.
2. Make sure WiFi is enabled on your laptop.
3. Check if your laptop detects any available networks.
4. If it detects your network, try forgetting the network and reconnecting by entering the password again.
5. Ensure your network drivers are